In [ ]:
import numpy as np
import pandas as pd

import time

# Data Cleaning

## Import raw data

In [ ]:
nov_2019 = pd.read_csv('On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2019_11.csv')

## Calculate average taxi in time for each destination airport and average taxi out time for ORD (origin airport)

> **NOTE: taxi times were eventually left out of the model**

In [ ]:
taxi_in_means = nov_2019.groupby('Dest')['TaxiIn'].mean()
taxi_in_means = taxi_in_means.apply(lambda x: pd.to_timedelta(x, unit='m').round('min'))
taxi_in_means.name = 'MeanTaxiIn'
taxi_in_means = taxi_in_means.reset_index()

ORD_taxi_out = pd.to_timedelta(nov_2019[nov_2019.Origin=='ORD'].TaxiOut.mean(), unit='m').round('min')

## Restrict data down to departures from Ohare

In [ ]:
ohare = nov_2019[(nov_2019.Origin=='ORD')].reset_index(drop=True)

# Drop cancelled flights
ohare = ohare[ohare.Cancelled == 0]

# Drop flights that were diverted and never got to destination
ohare = ohare[ohare.Diverted==0]
    
# Reformat times so I can parse to datetime
def add_zeros(s):
    while len(s) < 4:
        s = '0'+s
    return s

# Times are recorded with midnight=2400 instead of 0000 which is how python needs them
time_cols = ['CRSDepTime', 'DepTime', 'CRSArrTime', 'ArrTime']
for col in time_cols:
    
    ohare[col] = ohare[col].astype(int).astype(str)
    ohare[col] = ohare[col].replace('2400', '0000')
    ohare[col] = ohare[col].apply(add_zeros)
    
ohare = ohare.loc[:, :'FirstDepTime']

## Cut out unneeded columns

In [ ]:
flights = ohare.loc[:, ['FlightDate', 'Reporting_Airline', 'Flight_Number_Reporting_Airline',
                        'Tail_Number', 'Dest', 'CRSDepTime', 
                        'DepTime', 'DepDelayMinutes', 'CRSArrTime', 'ArrTime',
                        'ArrDelayMinutes', 'Distance']]

# Combine with average taxi times calculated above
flights = flights.merge(taxi_in_means, how='inner', on='Dest')
flights['MeanTaxiOut'] = ORD_taxi_out

flights.columns = ['date', 'airline', 'flight_no', 'tail_number', 'dest', 'sched_depart', 'depart', 'dep_delay',
                   'sched_arr', 'arr', 'arr_delay', 'distance', 'mean_taxi_in', 'mean_taxi_out']

## Parse dates/times

In [ ]:
flights['sched_depart'] = pd.to_datetime(flights.date + ' ' + flights.sched_depart)
flights['depart'] = pd.to_datetime(flights.date + ' ' + flights.depart)
flights['sched_arr'] = pd.to_datetime(flights.date + ' ' + flights.sched_arr)
flights['arr'] = pd.to_datetime(flights.date + ' ' + flights.arr)

flights['dep_delay'] = pd.to_timedelta(flights.dep_delay, unit='m')
flights['arr_delay'] = pd.to_timedelta(flights.arr_delay, unit='m')

## Fix dates/times to be in UTC Unix time

Times were recorded in the local time zone of each airport, so I converted all the times to UNIX time for standardization and easier calculations.

### Import airport data

In [ ]:
airports_all = pd.read_csv('AIRPORT_MASTER_RECORD.csv')
airports_all = airports_all.groupby('AIRPORT').first().reset_index()

# Filter down to airports in the Uniter States because I am only analyzing domestic flights
airports_all = airports_all[airports_all.AIRPORT_COUNTRY_NAME=='United States']

# Select only the destination airports in my dataset
airport_data = airports_all[airports_all.AIRPORT.isin(flights.dest)]

# Add ORD
airport_data = airport_data.append(airports_all[airports_all.AIRPORT=='ORD'])

airport_data = airport_data.set_index('AIRPORT')
airport_data.to_csv('airports.csv')

### Save time difference to UTC for each airport

In [ ]:
flights['ord_to_utc'] = pd.to_timedelta(airport_data.loc['ORD', 'UTC_LOCAL_TIME_VARIATION']/100, unit='h')
flights['dest_to_utc'] = pd.to_timedelta(airport_data.loc[flights.dest, 'UTC_LOCAL_TIME_VARIATION'].values/100, unit='h')

In [ ]:
flights['sched_dep_unix'] = (flights.sched_depart.dt.round('h') - flights.ord_to_utc).astype(int)/10**9
flights['sched_arr_unix'] = (flights.sched_arr.dt.round('h') - flights.dest_to_utc).astype(int)/10**9

## Filter out destinations that have <100 arrivals

Destination airports with fewer than 100 flights arriving do not provide enough data to be meaningful, and create noise within the data. By removing them I am able to focus modeling on busier airports in more populous areas.

In [ ]:
dest_mask = flights.groupby('dest').flight_no.count() > 100
flights = flights.set_index('dest').loc[dest_mask[dest_mask].index].reset_index()

## Make target column

My target column is a binary late arrival flag if a flight arrives any time after its scheduled arrival.

In [ ]:
flights['arr_late'] = (flights.arr_delay > pd.to_timedelta(0)).astype(int)
flights.to_csv('flights.csv', index=False)

In [ ]:
flights.head()

# Get weather data from DarkSky

I can use the Dark Sky API to fetch weather at each airport on an hourly basis. This weather data can be matched with the hour nearest to the arrival of a given flight.

## Create queries for every airport on every day of the month

In [ ]:
dates = pd.date_range('nov 1 2019', 'nov 30 2019').astype(str)
dates = dates+'T00:00:00'

with open('darksky_api_key.txt') as keyfile:
    darksky_key = keyfile.readline().strip()

# Build a DataFrame to store the query URL for each airport on each date
weather_queries = pd.DataFrame(columns=['Airport', 'Date', 'Query'])

# Retreive the latitude and longitude of each airport to build queries
for airport in flights.dest.unique():
    lat = airport_data.loc[airport, 'LATITUDE']
    lon = airport_data.loc[airport, 'LONGITUDE']
    for date in dates:
        q_dict = {
            'Airport':airport,
            'Date':date,
            'Query':f'https://api.darksky.net/forecast/{darksky_key}/{lat},{lon},{date}?exclude=currently'
        }
        weather_queries = weather_queries.append(q_dict, ignore_index=True)
        
# Add queries to find weather at my origin airport (ORD) for each date
for date in dates:
    q_dict = {
        'Airport':'ORD',
        'Date':date,
        'Query':f'https://api.darksky.net/forecast/{darksky_key}/{lat},{lon},{date}?exclude=currently'
    }
    weather_queries = weather_queries.append(q_dict, ignore_index=True)
    
weather_queries = weather_queries.set_index('Airport')

## Helper function to read weather for a given airport

In [ ]:
import requests

def get_weather(airport):
    df = pd.DataFrame()
    for i, q in enumerate(weather_queries.loc[airport, 'Query'], 1):
        r = requests.get(q)
        if r.status_code != 200:
            print(f'Error on request {q}')
            return None
        hourly = r.json()['hourly']['data']
        df = df.append(hourly)
    df['airport'] = airport
    return df

## Loop through airports and read all weather

> **NOTE: the DarkSky API only allows you to make 1,000 queries a day with a free API key, so if you intend on making more than 1,000 calls to the API at once you will need to register payment information with DarkSky**

In [ ]:
# Get weather data for arrival destinations
# weather_data = pd.DataFrame()
# for i, airport in enumerate(flight_data.Dest.unique()):
#     weather_data = weather_data.append(get_weather(airport))
#     weather_data.to_csv('weather_data.csv')
#     time.sleep(2)

# Get weather data for ORD
# ord_weather = get_weather('ORD')
# weather_data = weather_data.append(ord_weather)
# weather_data.to_csv('weather_data.csv', index=False)

# Send data to Postgres

In [ ]:
import psycopg2 as pg
import pandas.io.sql as psql
from sqlalchemy import create_engine

In [ ]:
weather_data = pd.read_csv('weather_data.csv')
flight_data = pd.read_csv('flights.csv')
airport_data = pd.read_csv('airports.csv')

In [ ]:
with open('aws_pgsql_pw.txt') as pwfile:
    password = pwfile.read()

params = {
    'host':'IP ADDRESS',
    'user':'USER',
    'dbname':'flight_delays',
    'port':5432,
    'password':password
}

connection_string = f'postgres://{params["user"]}:{params["host"]}@{params["host"]}:{params["port"]}/{params["dbname"]}'

engine = create_engine(connection_string)

I use the `to_sql` function only to send the schema of each of my tables to my Postgres server. The contents of these tables are copied into the database directly from csv files using the Postgres shell.

In [ ]:
flight_data.iloc[:0].to_sql('flights', engine, index=False)
weather_data.iloc[:0].to_sql('weather', engine, index=False)
airport_data.iloc[:0].to_sql('airports', engine, index=False)